In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/content/drive/My Drive/cs532_project2')

Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
    ):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
    ):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block,
        layers,
        num_classes=10,
        zero_init_residual=False,
        groups=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
    ):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                "or a 3-element tuple, got {}".format(replace_stride_with_dilation)
            )
        self.groups = groups
        self.base_width = width_per_group

        # CIFAR10: kernel_size 7 -> 3, stride 2 -> 1, padding 3->1
        self.conv1 = nn.Conv2d(
            3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False
        )
        # END

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(
            block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0]
        )
        self.layer3 = self._make_layer(
            block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1]
        )
        self.layer4 = self._make_layer(
            block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2]
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes,
                planes,
                stride,
                downsample,
                self.groups,
                self.base_width,
                previous_dilation,
                norm_layer,
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)

        return x


def _resnet(arch, block, layers, pretrained, progress, device, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = torch.load('/content/drive/My Drive/cs532_project2/resnet18.pt')
        # print(type(state_dict))
        # print(state_dict)
        # state_dict = torch.load(
        #     script_dir + "/state_dicts/" + arch + ".pt", map_location=device
        # )
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet18", BasicBlock, [2, 2, 2, 2], pretrained, progress, device, **kwargs
    )



In [ ]:
###Verify if gpu resource available for running the models and load the base pre-trained resnet-18 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=True)
model.load_state_dict(torch.load('/content/drive/My Drive/cs532_project2/resnet18.pt'))
model.to(device)

In [5]:
from torchvision import transforms as T
#Configuration of the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Values of hyper-parameters
num_epochs = 5
learning_rate = 0.0005

#Mean and standard deviation of the data calculated in order to normalize the data
mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)

transform = T.Compose(
            [
                T.ToTensor(),
                T.Normalize(mean, std),
            ]
        )

# Obtain the train and test datasets from CIFAR-10 dataset

train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transform)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1, 
                                          shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [6]:
import time as time

### Calculating the accuracy for the base model before pruning
num_correct = 0
num_samples = 0
before_prune_acc = 0
model.eval()
start = time.time()

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device=device)  # move to device, e.g. GPU
        y = y.to(device=device)
        scores = model(x)
        _, preds = scores.max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    end = time.time()
    before_prune_acc = float(num_correct) / num_samples
    # print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * before_prune_acc))

inference_time_per_image_without_pruning = (end - start)/num_samples

In [7]:
import torch.nn.utils.prune as prune

### Storing all the parameters from the model for pruning
parameters_to_prune = (

    (model.conv1, 'weight'),

    (model.layer1[0].conv1, 'weight'),
    (model.layer1[0].conv2, 'weight'),
    (model.layer1[1].conv1, 'weight'),
    (model.layer1[1].conv2, 'weight'),

    (model.layer2[0].conv1, 'weight'),
    (model.layer2[0].conv2, 'weight'),
    (model.layer2[1].conv1, 'weight'),
    (model.layer2[1].conv2, 'weight'),

    (model.layer3[0].conv1, 'weight'),
    (model.layer3[0].conv2, 'weight'),
    (model.layer3[1].conv1, 'weight'),
    (model.layer3[1].conv2, 'weight'),

    (model.layer4[0].conv1, 'weight'),
    (model.layer4[0].conv2, 'weight'),
    (model.layer4[1].conv1, 'weight'),
    (model.layer4[1].conv2, 'weight'),

    (model.fc, 'weight'),

)


### Storing all the modules so that the original weights that wwill be pruned and not required will be deleted
modules = []
for name, module in model.named_modules():
    if "conv" in name or "fc" in name:
        modules.append(module)

### Pruning function that will prune 50% of weights globally from the network
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,
)

In [8]:
#Cell to display the sparsity percentages as we go deeper in the neural network

print(
    "Sparsity in conv1: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer1->basicblock0->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer1[0].conv1.weight == 0))
        / float(model.layer1[0].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer1->basicblock0->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer1[0].conv2.weight == 0))
        / float(model.layer1[0].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer1->basicblock1->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer1[1].conv1.weight == 0))
        / float(model.layer1[1].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer1->basicblock1->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer1[1].conv2.weight == 0))
        / float(model.layer1[1].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer2->basicblock0->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer2[0].conv1.weight == 0))
        / float(model.layer2[0].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer2->basicblock0->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer2[0].conv2.weight == 0))
        / float(model.layer2[0].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer2->basicblock1->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer2[1].conv1.weight == 0))
        / float(model.layer2[1].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer2->basicblock1->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer2[1].conv2.weight == 0))
        / float(model.layer2[1].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer3->basicblock0->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer3[0].conv1.weight == 0))
        / float(model.layer3[0].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer3->basicblock0->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer3[0].conv2.weight == 0))
        / float(model.layer3[0].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer3->basicblock1->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer3[1].conv1.weight == 0))
        / float(model.layer3[1].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer3->basicblock1->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer3[1].conv2.weight == 0))
        / float(model.layer3[1].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer4->basicblock0->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer4[0].conv1.weight == 0))
        / float(model.layer4[0].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer4->basicblock0->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer4[0].conv2.weight == 0))
        / float(model.layer4[0].conv2.weight.nelement())
    )
)
print(
    "Sparsity in layer4->basicblock1->conv1: {:.2f}%".format(
        100. * float(torch.sum(model.layer4[1].conv1.weight == 0))
        / float(model.layer4[1].conv1.weight.nelement())
    )
)
print(
    "Sparsity in layer4->basicblock1->conv2: {:.2f}%".format(
        100. * float(torch.sum(model.layer4[1].conv2.weight == 0))
        / float(model.layer4[1].conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc: {:.2f}%".format(
        100. * float(torch.sum(model.fc.weight == 0))
        / float(model.fc.weight.nelement())
    )
)

Sparsity in conv1: 11.98%
Sparsity in layer1->basicblock0->conv1: 17.56%
Sparsity in layer1->basicblock0->conv2: 2.66%
Sparsity in layer1->basicblock1->conv1: 2.82%
Sparsity in layer1->basicblock1->conv2: 3.07%
Sparsity in layer2->basicblock0->conv1: 2.64%
Sparsity in layer2->basicblock0->conv2: 2.87%
Sparsity in layer2->basicblock1->conv1: 3.57%
Sparsity in layer2->basicblock1->conv2: 4.58%
Sparsity in layer3->basicblock0->conv1: 4.82%
Sparsity in layer3->basicblock0->conv2: 7.62%
Sparsity in layer3->basicblock1->conv1: 12.82%
Sparsity in layer3->basicblock1->conv2: 20.57%
Sparsity in layer4->basicblock0->conv1: 36.38%
Sparsity in layer4->basicblock0->conv2: 52.13%
Sparsity in layer4->basicblock1->conv1: 82.45%
Sparsity in layer4->basicblock1->conv2: 68.15%
Sparsity in fc: 0.04%


In [ ]:
import numpy as np
import time as time

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.7)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 200], gamma=0.1)

for mod in modules:
    prune.remove(mod, 'weight')

# def update_lr(optimizer, lr):    
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr


# total_step = len(train_loader)
# curr_lr = learning_rate

### Model Training
for epoch in range(num_epochs):
  # losses = []
  scheduler.step()
  start = time.time()

  for i, (inputs, targets) in enumerate(train_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      ## Forward Propogation and Loss calculation
      outputs = model(inputs)
      loss = criterion(outputs, targets)

      ## Backward Propogation and Optimization
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # losses.append(loss.item())

      end = time.time()
      
      if (i+1) % 100 == 0:
        print('Batch Index : %d Loss : %.4f Time : %.3f seconds ' % (
            i+1,
            loss.item(), 
            # np.mean(losses), 
            end - start))
    
        start = time.time()
        # if (i+1) % 100 == 0:
        #     print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
        #            .format(epoch+1, num_epochs, i+1, total_step, loss.item()))



In [ ]:
  ## Calculating the accuracy on the test set  
  num_correct = 0
  num_samples = 0
  model.eval()
  start = time.time()

  with torch.no_grad():
      for x, y in test_loader:
          x = x.to(device=device)  # move to device, e.g. GPU
          y = y.to(device=device)
          scores = model(x)
          _, preds = scores.max(1)
          num_correct += (preds == y).sum()
          num_samples += preds.size(0)

      end = time.time()
      acc = float(num_correct) / num_samples

      inference_time_per_image_with_pruning = (end - start)/num_samples

      print('Epoch : %d, Test Accuracy : %.3f %%' % (epoch, 100.*acc))
      print('Accuracy Drop : %.3f %%' % (100.*(before_prune_acc-acc)))
      print('Inference time per image before pruning : %.9f seconds' % (inference_time_per_image_without_pruning))
      print('Inference time per image after pruning : %.9f seconds' % (inference_time_per_image_with_pruning))
      print('Speedup : %.9f' % (inference_time_per_image_without_pruning/inference_time_per_image_with_pruning))


      print('--------------------------------------------------------------')
      model.train()   

In [17]:
## Saving the model to a .pt file
torch.save(model.state_dict(),'/content/drive/My Drive/cs532_project2/resnet18_prune_50perc_oneshot5.pt')